<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Julia_prog_language.svg/1200px-Julia_prog_language.svg.png" width=400 height=400 style="padding-right: 100px"/>

# Introducción

- Julia es un lenguaje dinámico de alto nivel y alta performance, diseñado para cómputo numérico
- El objetivo es tener un lenjuage expresivo y facil de usar sin sacrificar performance, y ser capaces de usar el mismo lenguaje en los entornos de prototipado y producción
- El primer release fue en 2012
- MIT License.
- La versión estable al momento es la 1.1.0

## Compilador

- Julia tiene un LLVM-based JIT compiler
- JIT description # TODO
- El código Julia es interpretado y compilado a native machine code.
- bytecode # TODO
- compare with JS, pypy # TODO
- bytecode vs llvm vs native code: https://stackoverflow.com/questions/43453944/what-is-the-difference-between-code-native-code-typed-and-code-llvm-in-julia

In [1]:
f(x) = x * x

f (generic function with 1 method)

In [2]:
code_llvm(f, (Float64,))


;  @ In[1]:1 within `f'
define double @julia_f_12561(double) {
top:
; ┌ @ float.jl:399 within `*'
   %1 = fmul double %0, %0
; └
  ret double %1
}


In [3]:
code_native(f, (Float64,))

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[1]:1 within `f'
; │┌ @ In[1]:1 within `*'
	vmulsd	%xmm0, %xmm0, %xmm0
; │└
	retl
	nopw	%cs:(%eax,%eax)
; └


# Features

- La sintáxis es basada en MATLAB y Python principalmente
- Combina features de lenguajes procedurales y funcionales, con algunos conceptos de POO
- Integra librerias de algebra lineal, estadística, procesamiento de señales, etc

- Tipado estático
- Inferencia de tipos
- Multiple-dispatch
- Las funciones son ciudadanos de primer clase

## Gestor de paquetes

In [ ]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("PyCall")
Pkg.add("RDatasets")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("PyPlot")
Pkg.add("StatsPlots")

## Integraciones con otros lenguajes

- C and Fortran pueden ser invocados via ccall.
- JavaCall
- PyCall
- RCall

### C

In [5]:
ccall((:clock, "libc"), Int32, ())

16712152

### PyCall

In [6]:
using PyCall
math = pyimport("math")
math.sin(math.pi / 4) - sin(pi / 4)

0.0

In [7]:
nr = pyimport("numpy.random")
nr.rand(3,4)

3×4 Array{Float64,2}:
 0.313863  0.664296  0.214447  0.233818
 0.697617  0.85155   0.318914  0.439609
 0.196488  0.733596  0.595428  0.850499

## Integración con la shell

In [8]:
run(`pwd`)

/Users/fede/dev/tmp/juliatalk


Process(`pwd`, ProcessExited(0))

## Vectores y matrices

In [9]:
x = rand(3,4)

3×4 Array{Float64,2}:
 0.879435  0.724004   0.0629187  0.595093
 0.46036   0.300761   0.504062   0.171858
 0.931749  0.0459905  0.232345   0.960193

### Indexing

In [10]:
x[1,:]

4-element Array{Float64,1}:
 0.879434573541696  
 0.7240035191572629 
 0.06291869001653239
 0.595092763176871  

In [11]:
x[:,1]

3-element Array{Float64,1}:
 0.879434573541696 
 0.4603601372488144
 0.9317492258017519

In [12]:
x[1,1]

0.879434573541696

- Indexing (en el lado derecho de una asignación) crea una copia del array original
- Asignaciones **no hacen** una copia

### Operaciones de algebra lineal

In [13]:
using LinearAlgebra

In [14]:
x'

4×3 Adjoint{Float64,Array{Float64,2}}:
 0.879435   0.46036   0.931749 
 0.724004   0.300761  0.0459905
 0.0629187  0.504062  0.232345 
 0.595093   0.171858  0.960193 

In [15]:
x2 = x * x'

3×3 Array{Float64,2}:
 1.65568   0.756595  1.43873 
 0.756595  0.586003  0.724905
 1.43873   0.724905  1.84623 

In [16]:
det(x2)

0.22956078144919856

In [17]:
pinv(x2)

3×3 Array{Float64,2}:
  2.42378  -1.54164   -1.2835  
 -1.54164   4.29869   -0.486468
 -1.2835   -0.486468   1.73286 

In [18]:
row = rand(4)

4-element Array{Float64,1}:
 0.18558447055772964
 0.583352492454192  
 0.7830115380175404 
 0.4960911331677764 

In [19]:
row'

1×4 Adjoint{Float64,Array{Float64,1}}:
 0.185584  0.583352  0.783012  0.496091

In [20]:
row' * row

1.2339552072409936

### Concatenación Horizontal y vertical

In [21]:
[1 2 3 4]

1×4 Array{Int64,2}:
 1  2  3  4

In [22]:
[1 ; 2 ; 3 ; 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [23]:
[1 2 ; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [24]:
[1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

### Operaciones vectoriales

In [25]:
x = rand(4,3)

4×3 Array{Float64,2}:
 0.519509  0.487485  0.754825
 0.607496  0.33845   0.317985
 0.29534   0.153198  0.756347
 0.950014  0.129762  0.367151

In [26]:
sum(x)

5.67756125453279

In [27]:
x^2

DimensionMismatch: DimensionMismatch("A has dimensions (4,3) but B has dimensions (4,3)")

In [28]:
x.^2

4×3 Array{Float64,2}:
 0.26989    0.237641   0.569761
 0.369051   0.114548   0.101115
 0.0872256  0.0234696  0.572061
 0.902526   0.0168382  0.134799

In [29]:
x .> 0.5

4×3 BitArray{2}:
  true  false   true
  true  false  false
 false  false   true
  true  false  false

In [30]:
x[x .> 0.5]

5-element Array{Float64,1}:
 0.5195090963511697
 0.6074960837652059
 0.9500136360532265
 0.7548253250801489
 0.7563469907696847

# Type system

Julia tiene distintas clases de tipos
- Tipos Primitivos como Int32, Int64, Bool, String Float64, etc (Se pueden definir nuevos tipos primitivos custom)
- Abstract types
- Types compuestos, mutables e immutables
- Tuple types
- Union types

## Types y herencia

- Los tipos compuestos son como structs o clases en otros lenguajes, y pueden contener valores de distinto tipo. Los tipos inmutables se declaran con el keyword "struct", y los mutables con el "mutable struct"
- En Julia, los tipos forman una jerarquía en donde solo los nodos hoja representan tipos concretos. No hay herencia de campos como en los lenguajes orientados a objetos.
- las "Interfaces" son definidas por colecciones de funciones que pueden ser implementadas para un cierto tipo para que se comporte de una determinada manera. Por ejemplo, si un tipo implementa las funciones "start", "next" y "done" se puede usar como un iterador (por ejemplo en un bloque for-in)
- Los tipos pueden ser parametricos, para habilitar "generic programming"

## Funciones

- Los operadores son funciones con sintaxis especial
- Dada una llamada a una función, Julia busca en tiempo de compilación a la definición mas específica basada en los tipos de los argumentos usados (Mas info en la sección de multiple dispatch)
- Estructuras mutables como arrays son pasados por referencia, lo que significa que no se realiza una copia y pueden ser alterados. Valores inmutables como integers son pasados por valor

## Funciones de alto orden y funciones anónimas

In [31]:
x = collect(1:10)

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [32]:
map(e -> e ^ 2, x)

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [33]:
filter(e -> e % 2 == 0, x)

5-element Array{Int64,1}:
  2
  4
  6
  8
 10

In [34]:
foldl(+, x)

55

## Ejemplo

- Definición de tipos
- Declaración de funciones
- Tipos paramétricos
- Multiple dispatch (explicado más adelante)
- Method extension
- Interpolación de Strings

In [35]:
abstract type Tree{T} end
struct Node{T} <: Tree{T} 
    left::Tree{T}
    right::Tree{T}
end
struct Leaf{T} <: Tree{T}
    value::T
end
struct Empty{T} <: Tree{T} end

# El tipo puede ser omitido si no es necesario
height(t::Node) = max(height(t.left), height(t.right)) + 1
height(t::Leaf) = 0
height(t::Empty) = -1

# El metodo size de Base tiene que ser importado explícitamente para ser extendido
Base.size(t::Node) = size(t.left) + size(t.right) + 1
Base.size(t::Leaf) = 1
Base.size(t::Empty) = 0

# Notar que las funciones pueden ser declaradas en forma compacta para expresiones de una linea
show(t::Tree) = show(t, 0)
show(t::Leaf, margin::Int) = println("$(repeat(" ", margin))Leaf($(t.value))")
show(t::Empty, margin::Int) = println("$(repeat(" ", margin))-")
function show(t::Node, margin::Int)
    println("$(repeat(" ", margin))Node")
    show(t.left, margin + 4)
    show(t.right, margin + 4)
end


tree = Node(Node(Leaf(1), Leaf(2)), Node(Leaf(3), Leaf(4)))
println(typeof(tree))
println("height = $(height(tree))")
println("size = $(size(tree))")
show(tree)

Node{Int64}
height = 2
size = 7
Node
    Node
        Leaf(1)
        Leaf(2)
    Node
        Leaf(3)
        Leaf(4)


## Inmutabilidad

Structs son immutables por default
Si querés structs mutables podés obtenerlos mediante `mutable struct`

Un objeto con un tipo inmutable es pasado por copia, mientras en un tipo inmutable es pasado por referencia

Como regla general, si dos objetos del mismo tipo con los mismos valores para los mismos campos se pueden considerar como idénticos, entonces el tipo deberia ser definido como inmutable. En caso contrario, si deberian ser considerados diferentes objetos porque pueden cambiar independientemente, entonces el tipo debería ser mutable

## Multiple dispatch

- En lenguajes de single-dispatch, el método invocado depende (en tiempo de ejecución) del objeto dueño del mismo. "Objects own methods"

```python
class Dog:
    def shout(self):
        print("GUAU")

class Cat:
    def shout(self):
        print("MIAU")

animals = [Dog(), Cat()]
for animal in animals:
    animal.shout()
```

- Double-dispatch (resolviendo el método basado en los tipos de dos objetos) se puede simular haciendo varias llamadas a funciones (visitor pattern)

```python
class A:
    def f(self):
        print("A")

    def doSomething(self, other):
        other.doSomethingWithA(self)

    def doSomethingWithA(self, other):
        print("AA")

    def doSomethingWithB(self, other):
        print("AB")

class B:
    def f(self):
        print("B")

    def doSomething(self, other):
        other.doSomethingWithB(self)

    def doSomethingWithA(self, other):
        print("BA")

    def doSomethingWithB(self, other):
        print("BB")

xs = [A(), B()]
for x in xs:
    for y in xs:
        x.doSomething(y)
```

- Multiple dispatch consiste en determinar el metodo correct a ser invocado basado en los tipos de TODOS los argumentos involucrados, no sólo el primer
- En Julia, los objetos **NO** son los dueños de los métodos que operan sobre ellos
- En cambio, distintas funciones con el mismo nombre proveen distintos comportamientos para diferentes tipos. Estas diferentes implementaciones se llaman "métodos" de la misma "función". Esto es análogo a los conceptos de mensajes (un pedido por una operación) y métodos (una implementación concreta de ese pedido) de los lenguajes orientados a objetos

In [36]:
methods(size)

# 101 methods for generic function "size":
[1] size(B::BitArray{1}) in Base at bitarray.jl:77
[2] size(B::BitArray{1}, d) in Base at bitarray.jl:81
[3] size(r::Core.Compiler.StmtRange) in Base.IRShow at show.jl:1561
[4] size(itr::Base.AsyncGenerator) in Base at asyncmap.jl:409
[5] size(zmsg::ZMQ.Message) in ZMQ at /Users/fede/.julia/packages/ZMQ/ABGOx/src/message.jl:95
[6] size(b::PyBuffer) in PyCall at /Users/fede/.julia/packages/PyCall/ttONZ/src/pybuffer.jl:68
[7] size(b::PyBuffer, d::Integer) in PyCall at /Users/fede/.julia/packages/PyCall/ttONZ/src/pybuffer.jl:74
[8] size(o::PyObject) in PyCall at /Users/fede/.julia/packages/PyCall/ttONZ/src/PyCall.jl:806
[9] size(g::Base.Generator) in Base at generator.jl:51
[10] size(t::Tuple, d) in Base at tuple.jl:22
[11] size(S::Base.Slice) in Base at indices.jl:328
[12] size(S::Base.IdentityUnitRange) in Base at indices.jl:360
[13] size(r::AbstractRange) in Base at range.jl:470
[14] size(x::Number) in Base at number.jl:62
[15] size(x::Number, d) in Base at number.jl:63
[16] size(x::Ref) in Base at refpointer.jl:38
[17] size(iter::LinearIndices) in Base at indices.jl:442
[18] size(a::Array{T,1} where T) in Base at array.jl:155
[19] size(a::Array{T,2} where T) in Base at array.jl:156
[20] size(a::Array{#s72,N} where #s72) where N in Base at array.jl:157
[21] size(a::Array, d) in Base at array.jl:154
[22] size(V::SubArray) in Base at subarray.jl:65
[23] size(A::Base.ReshapedArray) in Base at reshapedarray.jl:205
[24] size(a::Base.ReinterpretArray{T,0,S,A} where A<:AbstractArray{S,0} where S) where T in Base at reinterpretarray.jl:77
[25] size(a::Base.ReinterpretArray{T,N,S,A} where A<:AbstractArray{S,N} where N) where {T, S} in Base at reinterpretarray.jl:73
[26] size(B::BitArray) in Base at bitarray.jl:78
[27] size(s::Base.CodeUnits) in Base at strings/basic.jl:695
[28] size(iter::CartesianIndices) in Base.IteratorsMD at multidimensional.jl:356
[29] size(L::Base.LogicalIndex) in Base at multidimensional.jl:541
[30] size(A::PermutedDimsArray{T,N,perm,iperm,AA} where AA<:AbstractArray where iperm) where {T, N, perm} in Base.PermutedDimsArrays at permuteddimsarray.jl:49
[31] size(a::Random.UnsafeView) in Random at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Random/src/RNGs.jl:382
[32] size(v::Union{Adjoint{T,#s623}, Transpose{T,#s623}} where #s623<:(AbstractArray{T,1} where T) where T) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/adjtrans.jl:123
[33] size(A::Union{Adjoint{T,#s623}, Transpose{T,#s623}} where #s623<:(AbstractArray{T,2} where T) where T) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/adjtrans.jl:124
[34] size(A::SymTridiagonal) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/tridiag.jl:113
[35] size(A::SymTridiagonal, d::Integer) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/tridiag.jl:115
[36] size(M::Tridiagonal) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/tridiag.jl:466
[37] size(M::Tridiagonal, d::Integer) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/tridiag.jl:468
[38] size(A::LowerTriangular) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/triangular.jl:37
[39] size(A::LowerTriangular, d) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/triangular.jl:36
[40] size(A::UnitLowerTriangular) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/triangular.jl:37
[41] size(A::UnitLowerTriangular, d) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/j

En una llamada a una función, Julia busca en tiempo de compilación por la definición mas específica, es decir aquella cuyos tipos son mas cercanos a los tipos de los argumentos con los que se invoca, y arrojará un error en caso de que no encuentre ninguna
Multiple dispatch nos permite escribir código más conciso, y permite definir comportamiento para nuevos tipos de una forma más sencilla

In [37]:
struct A end
struct B end

doSomething(x::A, y::A) = println("AA")
doSomething(x::A, y::B) = println("AB")
doSomething(x::B, y::A) = println("BA")
doSomething(x::B, y::B) = println("BB")

xs = [A(), B()]
for x in xs
    for y in xs
        doSomething(x,y)
    end
end

AA
AB
BA
BB


## DataFrames

In [38]:
using Pkg
using DataFrames

┌ Info: Recompiling stale cache file /Users/fede/.julia/compiled/v1.1/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1184


In [ ]:
using RDatasets;
df = dataset("datasets", "iris")

┌ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1186


In [ ]:
size(df)

In [ ]:
first(df, 5)

In [ ]:
last(df, 5)

In [ ]:
names(df)

In [ ]:
df[:Species] # or df.Species

In [ ]:
describe(df)

Seleccionando columnas:

In [ ]:
df[:, 2]

seleccionando filas:

In [ ]:
df[1, :]

Boolean indexing

In [ ]:
df[:Species] .== "setosa"

In [ ]:
df[df[:Species] .== "setosa", :]

In [ ]:
groupby(df, :Species)

Split-apply-combine en una sola linea

In [ ]:
using Statistics
mean_petal_lengths = by(df, :Species, mean = :PetalLength => mean, count = :PetalLength => length)

# Plotting

In [ ]:
using Plots, StatsPlots

In [ ]:
backends()

In [ ]:
pyplot()

In [ ]:
@df df scatter(:SepalLength, :SepalWidth, group=:Species,
        title = "My awesome plot",
        xlabel = "Length", ylabel = "Width",
        m=(0.5, [:cross :hex :star7], 12),
        bg=RGB(.2,.2,.2))

In [ ]:
# define the Lorenz attractor
mutable struct Lorenz
    dt; σ; ρ; β; x; y; z
end

function step!(l::Lorenz)
    dx = l.σ*(l.y - l.x)       ; l.x += l.dt * dx
    dy = l.x*(l.ρ - l.z) - l.y ; l.y += l.dt * dy
    dz = l.x*l.y - l.β*l.z     ; l.z += l.dt * dz
end

attractor = Lorenz((dt = 0.02, σ = 10., ρ = 28., β = 8//3, x = 1., y = 1., z = 1.)...)


# initialize a 3D plot with 1 empty series
plt = plot3d(1, xlim=(-25,25), ylim=(-25,25), zlim=(0,50),
                title = "Lorenz Attractor", marker = 2)

# build an animated gif by pushing new points to the plot, saving every 10th frame
@gif for i=1:1500
    step!(attractor)
    push!(plt, attractor.x, attractor.y, attractor.z)
end every 10

# TLDR:

- Lenguaje de alto nivel
- Alta performance, comparable con C
- Diseñado para el cálculo numérico y la comunidad científica
- Interoperabilidad con Python, C
- Static typing, Type inference
- Multidimensional arrays "numpy built-in"
- Multiple-dispatch
- Muchos paquetes de Data Science creados por la comunidad científica

# Repo

https://github.com/marcosfede/julia_talk

<img src="repo.png" width=400 height=400/>

## Links

https://julialang.org/

Tutorials:  
http://ucidatascienceinitiative.github.io/IntroToJulia/  
https://github.com/scidom/StatsLearningByExample.jl  
http://www.breloff.com/JuliaML-and-Plots/  
https://lectures.quantecon.org/jl/julia_plots.html#plotsjl-jl  
https://en.wikibooks.org/wiki/Introducing_Julia/Plotting

Articles:  
http://ucidatascienceinitiative.github.io/IntroToJulia/Html/WhyJulia  
http://www.stochasticlifestyle.com/7-julia-gotchas-handle/  
https://arstechnica.com/science/2014/05/scientific-computings-future-can-any-coding-language-top-a-1950s-behemoth/  
http://www.admin-magazine.com/HPC/Articles/Julia-A-New-Language-For-Technical-Computing  
http://www.admin-magazine.com/HPC/Articles/Parallel-Julia-Jumping-Right-In  
https://www.evanmiller.org/why-im-betting-on-julia.html  